In [2]:
!pip install --user --upgrade pip

In [3]:
! pip install numpy pandas plotly scipy matplotlib

In [4]:
! pip install numpy openpyxl

In [5]:
import numpy as np
import os
import pandas as pd
from datetime import datetime
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import tarfile
import zipfile
from scipy.io import loadmat
from urllib.request import urlretrieve
from os.path import isfile, isdir
import openpyxl
import plotly.express as px
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [6]:
df = pd.read_excel("data/resultados_pruebas_saber_2020-1.xlsx")

In [7]:
df.describe(include = 'all')

,ESTU_TIPODOCUMENTO,ESTU_NACIONALIDAD,ESTU_GENERO,ESTU_FECHANACIMIENTO,ESTU_EXTERIOR,PERIODO,ESTU_CONSECUTIVO,ESTU_ESTUDIANTE,ESTU_PAIS_RESIDE,ESTU_DISC_FISICA,ESTU_DISC_SISTEM,ESTU_DISC_AUDITCASTELL,ESTU_DISC_AUDITLSC,ESTU_DISC_VISUALCEGUE,ESTU_DISC_VISUALBAJA,ESTU_DISC_VOZHABLA,ESTU_DISC_INTELEC,ESTU_DISC_ESPAUTISTA,ESTU_DISC_PSICOSOCIAL,ESTU_DISC_ASPERGER,ESTU_DEPTO_RESIDE,ESTU_COD_RESIDE_DEPTO,ESTU_MCPIO_RESIDE,ESTU_COD_RESIDE_MCPIO,ESTU_AREARESIDE,ESTU_ESTADOCIVIL,ESTU_COLE_TERMINO,ESTU_CODDANE_COLE_TERMINO,ESTU_COD_COLE_MCPIO_TERMINO,ESTU_OTROCOLE_TERMINO,ESTU_TITULOOBTENIDOBACHILLER,ESTU_PAGOMATRICULABECA,ESTU_PAGOMATRICULACREDITO,ESTU_PAGOMATRICULAPADRES,ESTU_PAGOMATRICULAPROPIO,ESTU_COMOCAPACITOEXAMENSB11,ESTU_TIPODOCUMENTOSB11,ESTU_SEMESTRECURSA,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,FAMI_TRABAJOLABORPADRE,FAMI_TRABAJOLABORMADRE,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_TIENESERVICIOTV,FAMI_TIENECOMPUTADOR,FAMI_TIENELAVADORA,FAMI_TIENEHORNOMICROOGAS,FAMI_TIENEAUTOMOVIL,FAMI_TIENEMOTOCICLETA,FAMI_TIENECONSOLAVIDEOJUEGOS,FAMI_CUANTOSCOMPARTEBAÑO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,INST_COD_INSTITUCION,INST_NOMBRE_INSTITUCION,ESTU_PRGM_ACADEMICO,ESTU_SNIES_PRGMACADEMICO,GRUPOREFERENCIA,ESTU_PRGM_CODMUNICIPIO,ESTU_PRGM_MUNICIPIO,ESTU_PRGM_DEPARTAMENTO,ESTU_NIVEL_PRGM_ACADEMICO,ESTU_METODO_PRGM,ESTU_NUCLEO_PREGRADO,ESTU_INST_CODMUNICIPIO,ESTU_INST_MUNICIPIO,ESTU_INST_DEPARTAMENTO,INST_CARACTER_ACADEMICO,INST_ORIGEN,ESTU_PRIVADO_LIBERTAD,ESTU_COD_MCPIO_PRESENTACION,ESTU_MCPIO_PRESENTACION,ESTU_DEPTO_PRESENTACION,ESTU_COD_DEPTO_PRESENTACION,MOD_RAZONA_CUANTITAT_PUNT,MOD_RAZONA_CUANTITATIVO_PNAL,MOD_RAZONA_CUANTITATIVO_PNBC,MOD_LECTURA_CRITICA_PUNT,MOD_LECTURA_CRITICA_PNAL,MOD_LECTURA_CRITICA_PNBC,MOD_COMPETEN_CIUDADA_PUNT,MOD_COMPETEN_CIUDADA_PNAL,MOD_COMPETEN_CIUDADA_PNBC,MOD_INGLES_PUNT,MOD_INGLES_DESEM,MOD_INGLES_PNAL,MOD_INGLES_PNBC,MOD_COMUNI_ESCRITA_PUNT,MOD_COMUNI_ESCRITA_DESEM,MOD_COMUNI_ESCRITA_PNAL,MOD_COMUNI_ESCRITA_PNBC,PUNT_GLOBAL,PERCENTIL_GLOBAL,PERCENTIL_NBC,ESTU_ESTADOINVESTIGACION
count,68725,68725,68597,68667,68725,68725.0,68725,68725,68725,27,3,8,8,1,10,1,3,1,2,2,68688,68688.000000,68688,68688.00000,58499,58190,37609,3.760900e+04,37609.000000,11543,58465,58461,58461,58463,58461,58475,57799,58465.0,65210,65100,56760,60492,64573,64460,64325,64056,64006,63846,63817,63840,63783,63863,63706,63606,68725.000000,68725,68725,68661.000000,68725,68725.00000,68725,68725,68725,68725,68725,68725.000000,68725,68725,68725,68725,68725,68725.000000,68725,68725,68725.000000,68725.000000,68567.000000,68566.000000,68725.000000,68567.000000,68566.000000,68725.000000,68567.000000,68566.000000,68705.000000,52346,68567.000000,68566.000000,67851.000000,58108.000000,67696.000000,67695.000000,68725.000000,68567.000000,68566.000000,68725
unique,6,17,2,10735,1,NaN,68725,1,17,1,1,1,1,1,1,1,1,1,1,1,34,NaN,913,NaN,2,4,7201,NaN,NaN,9655,3,2,2,2,2,3,5,12.0,12,12,13,13,7,2,2,2,2,2,2,2,2,6,7,5,NaN,184,982,NaN,20,NaN,136,32,2,3,50,NaN,51,25,5,6,1,NaN,3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,CC,COLOMBIA,M,2000-09-21 00:00:00,NO,NaN,EK202010002443,ESTUDIANTE,COLOMBIA,x,x,x,x,x,x,x,x,x,x,x,BOGOTÁ,NaN,BOGOTÁ D.C.,NaN,Cabecera Municipal,Soltero,INST EDUC CEFA,NaN,NaN,otro,Bachiller académico,No,No,No,No,Repasó por cuenta propia,TI,4.0,Primaria incompleta,Secundaria (Bachillerato) completa,Trabaja por cuenta propia (por ejemplo plomero...,"Trabaja en el hogar, no trabaja o estudia",Estrato 2,Si,Si,Si,Si,No,No,No,No,3 o 4,No pago semestre,Más de 30 horas,NaN,SERVICIO NACIONAL DE APRENDIZAJE-SENA,TECNOLOGÍA EN CONTABILIDAD Y FINANZAS,NaN,TECNOLÓGICO EN ADMINISTRACIÓN Y TURISMO,NaN,BOGOTÁ D.C.,BOGOTÁ,TECNOLOGÍA,PRESENCIAL,ADMINISTRACIÓN,NaN,BOGOTÁ D.C.,BOGOTÁ,INSTITUCIÓN TECNOLÓGICA,OFICIAL NACIONAL,N,NaN,BOGOTÁ D.C.,BOGOTÁ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PUBLICAR
freq,67218,68663,34798,45,68725,NaN,1,68725,68663,27,3,8,8,1,10,

# Transformación de los Datos

### Variable: ESTU_TIPODOCUMENTO

In [8]:
df[['ESTU_TIPODOCUMENTO','PUNT_GLOBAL']].groupby("ESTU_TIPODOCUMENTO").agg(["count", "mean"])

PUNT_GLOBAL            
                         count        mean
ESTU_TIPODOCUMENTO                        
CC                       67218   93.080708
CE                          63  100.857143
CR                         125   98.200000
PE                           4  121.250000
PEP                          2   99.000000
TI                        1313   98.118812

### Descartado

Se descarta la variable ESTU_TIPODOCUMENTO, porque la gran mayoria de los estudiantes tienen una cedula, por lo que no representa un impacto en el resultado del puntaje global

### Variable: ESTU_NACIONALIDAD

In [9]:
df[['ESTU_NACIONALIDAD','PUNT_GLOBAL']].groupby("ESTU_NACIONALIDAD").agg(["count", "mean"])

PUNT_GLOBAL            
                           count        mean
ESTU_NACIONALIDAD                           
ALEMANIA                       1  135.000000
ANTIGUA Y BARBUDA              1  108.000000
BOLIVIA                        2  107.500000
CANADÁ                         1   88.000000
CHILE                          1  103.000000
COLOMBIA                   68663   93.185471
ECUADOR                        2  118.000000
EL SALVADOR                    4  138.750000
ESPAÑA                         2   96.500000
ESTADOS UNIDOS                 2   82.500000
HAITI                          2   80.500000
MÉXICO                         2  113.000000
NICARAGUA                      2   81.500000
PANAMÁ                         8   89.625000
PERÚ                           2  109.500000
REPÚBLICA DOMINICANA           2  122.500000
VENEZUELA                     28  104.214286

### Descartado

Se descarta la variable ESTU_NACIONALIDAD, porque la gran mayoria de los estudiantes son de Colombia, por lo que no representa un impacto en el resultado del puntaje global.

### Variable: ESTU_GENERO

In [10]:
df[['ESTU_GENERO','PUNT_GLOBAL']].groupby("ESTU_GENERO").agg(["count", "mean"])

PUNT_GLOBAL           
                  count       mean
ESTU_GENERO                       
F                 33799  92.733335
M                 34798  93.646474

### Descartado

Se descarta la variable ESTU_GENERO, porque tanto hombres como mujeres sacaron casi la misma media de puntuación global en las pruebas saber, por lo tanto, no afecta al resultado final que sea hombre o mujer.

### Variable: ESTU_FECHANACIMIENTO

In [16]:
hoy = datetime.today()

df['ESTU_FECHANACIMIENTO'] = pd.to_datetime(df['ESTU_FECHANACIMIENTO'], format='%Y-%m-%d', errors='coerce')

df['ESTU_EDAD'] = (hoy - pd.to_datetime(df['ESTU_FECHANACIMIENTO'], format='%Y-%m-%d')).astype('<m8[Y]')

edad_media = df['ESTU_EDAD'].mean()
mask = (df['ESTU_EDAD'] < 10) | (df['ESTU_EDAD'] > 100)
df.loc[mask, 'ESTU_EDAD'] = edad_media

df.to_excel("data/resultados_pruebas_saber_2020-1.xlsx", index=False)

In [17]:
df[['ESTU_EDAD','PUNT_GLOBAL']].groupby("ESTU_EDAD").agg(["count", "mean"])

PUNT_GLOBAL            
                count        mean
ESTU_EDAD                        
12.000000           1   45.000000
13.000000           1  104.000000
18.000000           5  111.200000
19.000000          31  102.483871
20.000000         502   99.031873
21.000000        3533   96.538070
22.000000        9728   94.166632
23.000000        8363   92.537008
24.000000        6309   91.904422
25.000000        4861   91.821847
26.000000        4147   92.448999
27.000000        3584   92.482701
28.000000        3070   92.821173
28.270291          31   91.838710
29.000000        2706   93.344789
30.000000        2385   93.919078
31.000000        2086   94.234899
32.000000        1971   94.010147
33.000000        1796   93.944321
34.000000        1597   93.815905
35.000000        1371   93.574034
36.000000        1224   93.472222
37.000000        1113   92.513926
38.000000        1029   93.078717
39.000000         922   94.355748
40.000000         865   92.094798
41.000000         840   93.540476
42.000000         672   93.720238
43.000000         663   93.911011
44.000000         480   93.014583
45.000000         404   93.113861
46.000000         376   92.026596
47.000000         285   93.540351
48.000000         270   91.155556
49.000000         226   90.181416
50.000000         198   88.540404
51.000000         181   87.381215
52.000000         140   91.142857
53.000000         127   92.685039
54.000000          94   88.531915
55.000000         108   90.777778
56.000000          61   85.065574
57.000000          56   84.750000
58.000000          56   91.107143
59.000000          49   87.632653
60.000000          32   86.781250
61.000000          25   94.080000
62.000000          23   88.043478
63.000000           9   81.000000
64.000000          16   86.000000
65.000000           8   94.250000
66.000000           3   93.333333
67.000000           5   78.400000
68.000000           3   67.333333
69.000000           4   75.000000
70.000000           4   98.750000
71.000000           2   72.500000
83.000000           1  115.000000

### Variable: ESTU_EXTERIOR

In [18]:
df[['ESTU_EXTERIOR','PUNT_GLOBAL']].groupby("ESTU_EXTERIOR").agg(["count", "mean"])

PUNT_GLOBAL           
                    count       mean
ESTU_EXTERIOR                       
NO                  68725  93.195213

### Descartado

Se descarta la variable ESTU_EXTERIOR, porque nadie estudio en el exterior, por lo tanto no afeca al puntaje global.

In [9]:
df.drop(labels=['ESTU_TIPODOCUMENTO', 'ESTU_NACIONALIDAD', 'ESTU_GENERO', 'ESTU_FECHANACIMIENTO'], axis=1, inplace=True)